<div class='bar_title'></div>

*Decision Support Systems*

# PuLP Introduction

Toni Greif<br>
Chair of Information Systems and Management

Winter Semester 19/20

Before each PuLP implementation the package must be installed (not necessary if you don't use the Colab enviroment) and imported.

In [3]:
!pip install pulp
from pulp import *
import pandas as pd

## Example

A chocolate manufacturing company produces two types of chocolate – A and B. Both the chocolates require Milk and Choco only,


- each unit of Milk costs \\$1 and each unit of Choco \\$1.5
- each unit of A requires 1 unit of Milk and 3 units of Choco,
- each unit of B requires 2 unit of Milk and 5 units of Choco.

The company has a total of 5 units of Milk and 12 units of Choco.

The company sells
- each unit of A for \\$9
- each unit of B for \\$7

How many units of A and B should the company produce  to maximize its profit?

Our typical workflow will be the following:
- Read the text carefully and try to identify the following elements
    1. parameters
    - decision variables
    - objective
    - constraints
    
- Try to formulate the complete optimization problem by hand

- Implement your model with the following steps
    1. create the model
    - set the parameters
    - create the decision variables
    - set the objective
    - set the constraints
    - solve the problem
    - print the solution

So let's start with our first model...

In [295]:
# Create model
prob = LpProblem("chocolate maufacturing", sense=pulp.LpMaximize)

In [296]:
# Set parameters
## Creates a list of the chocolates and ingredients
chocolates = ['A', 'B']
ingredients = ['Milk', 'Choco']

## Dictionaries of the costs, prices, milk and choco protion, maximal Ingredients are created
costs = {'Milk': 0.1, 
         'Choco': 0.3}

prices = {'A': 9,
          'B': 14}

portion = {'Milk': {'A': 1, 'B': 2},
           'Choco': {'A': 3,'B': 2}}

maxIngredients = {'Milk': 20, 
                  'Choco': 40}

In [297]:
# Create variables
chocolates_vars = LpVariable.dicts("chocolates_production", chocolates, lowBound=0, cat='Continuous')

In [298]:
# Set objective
prob += lpSum([chocolates_vars[chocolate] * prices[chocolate] for chocolate in chocolates]
              +[chocolates_vars[chocolate] * portion[ingredient][chocolate] * -costs[ingredient]
                 for chocolate in chocolates for ingredient in ingredients])

In [299]:
# Set constraints
for ingredient in ingredients:
    prob += lpSum(chocolates_vars[chocolate] * portion[ingredient][chocolate]
                  for chocolate in chocolates) <= maxIngredients[ingredient]

The quantifiers have become `` for...in loops`` and the summations have become calls to ``lpSum``

In [300]:
prob

chocolate maufacturing:
MAXIMIZE
8.0*chocolates_production_A + 13.200000000000001*chocolates_production_B + 0.0
SUBJECT TO
_C1: chocolates_production_A + 2 chocolates_production_B <= 20

_C2: 3 chocolates_production_A + 2 chocolates_production_B <= 40

VARIABLES
chocolates_production_A Continuous
chocolates_production_B Continuous

In [301]:
# Solve problem
prob.solve()
LpStatus[prob.status]

'Optimal'

In [302]:
for variable in prob.variables():
    print ("{} = {}".format(variable.name, variable.varValue))

chocolates_production_A = 10.0
chocolates_production_B = 5.0


In [303]:
print (pulp.value(prob.objective))

146.0


In [304]:
o = [{'name':v.name, 'reduced costs':v.dj} for v in prob.variables()]
print(pd.DataFrame(o))

                      name  reduced costs
0  chocolates_production_A           -0.0
1  chocolates_production_B           -0.0


In [305]:
o = [{'name':name, 'shadow price':c.pi, 'slack': c.slack} for name, c in prob.constraints.items()]
print(pd.DataFrame(o))

  name  shadow price  slack
0  _C1           5.9   -0.0
1  _C2           0.7   -0.0
